# CUET CSE FEST 2025
## PoliMemeDecode: Humor That Speaks Politics

**Competition:** Political Meme Classification Challenge  
**Task:** Classify Bangla memes as **Political** or **NonPolitical**

**Dataset:** https://www.kaggle.com/datasets/shahriar26s/meme-dataset

---

### Two-Stage Classification Pipeline
- **Stage 1:** Qwen2.5-VL-7B-Instruct (4-bit quantized) for initial classification
- **Stage 2:** Phi-3-Vision-128k-Instruct for keyword-based validation

---

### Install Dependencies
Install required libraries: transformers, bitsandbytes for quantization, and image processing tools.

In [1]:
# Cell 1: Install Required Libraries
# Use bitsandbytes for 4-bit quantization to fit on T4 GPU
!pip install transformers accelerate bitsandbytes -q
!pip install pandas pillow tqdm -q
!pip install qwen-vl-utils -q
print("Libraries installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's de

### Import Libraries
Import PyTorch, transformers, pandas, PIL for image handling, and check GPU availability.

In [2]:
# Cell 2: Import Libraries
import torch
import pandas as pd
import os
import gc
import json
import re
from PIL import Image
from tqdm import tqdm
from datetime import datetime
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig

# Check GPU
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU Memory: {gpu_mem:.1f} GB")

print("\nLibraries imported successfully!")

2025-12-05 09:26:45.748729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764926805.978552      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764926806.094383      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

GPU Available: True
GPU Name: Tesla P100-PCIE-16GB
GPU Memory: 17.1 GB

Libraries imported successfully!


### Define Paths
Configure paths for test images, annotated CSV dataset, and output files.

In [ ]:
# Cell 3: Define Paths
# =============================================
# UPDATE THESE PATHS ACCORDING TO YOUR UPLOAD
# =============================================

# Test images from competition dataset
IMAGE_DIR = "/kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Test/Image"

# Your custom uploaded CSV with annotations
# CHANGE THIS PATH to match your custom dataset upload name
ANNOTATED_CSV_PATH = "/kaggle/input/meme-dataset/merged_meme_data.csv"

# Output paths
OUTPUT_DIR = "/kaggle/working"
SUBMISSION_PATH = f"{OUTPUT_DIR}/submission.csv"
CHECKPOINT_PATH = f"{OUTPUT_DIR}/prediction_checkpoint.csv"
STAGE2_CHECKPOINT_PATH = f"{OUTPUT_DIR}/stage2_checkpoint.csv"

print("Paths configured:")
print(f"   Images: {IMAGE_DIR}")
print(f"   CSV: {ANNOTATED_CSV_PATH}")
print(f"   Output: {SUBMISSION_PATH}")

Paths configured:
   Images: /kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Test/Image
   CSV: /kaggle/input/meme-dataset/merged_meme_data.csv
   Output: /kaggle/working/submission.csv


### Load Dataset
Load annotated CSV with Bangla text encoding and display dataset statistics.

In [4]:
# Cell 4: Load and Explore CSV Data
print("Loading annotated CSV...")

# Load with UTF-8-sig encoding for Bangla text
try:
    df = pd.read_csv(ANNOTATED_CSV_PATH, encoding='utf-8-sig')
except:
    df = pd.read_csv(ANNOTATED_CSV_PATH, encoding='utf-8')

print(f"\nDataset Info:")
print(f"   Total records: {len(df)}")
print(f"   Columns: {df.columns.tolist()}")

print(f"\nFirst 5 rows:")
display(df.head())

print(f"\nColumn Statistics:")
for col in df.columns:
    non_null = df[col].notna().sum()
    print(f"   {col}: {non_null}/{len(df)} non-null")

Loading annotated CSV...

Dataset Info:
   Total records: 330
   Columns: ['Image_name', 'extracted_text', 'Language', 'Humor', 'Metaphor', 'Meme_Explanation', 'Metaphor_Object', 'Political_Intensity']

First 5 rows:


,Image_name,extracted_text,Language,Humor,Metaphor,Meme_Explanation,Metaphor_Object,Political_Intensity
0,test0001.jpg,*Looses Ducsu Election\n- Any comment on the e...,English,Ironic,Both,This meme satirizes Bangladeshi politician Nur...,Ducsu Election,High
1,test0002.jpg,kya matlab ke fue,Banglish,Mockery,Both,This reaction meme features an image of a disa...,Relatable social reaction,Low
2,test0003.jpg,SNACKS MUJIB DURING\n15TH AUG SEHR!,Banglish,Mockery,Both,This meme satirizes individuals who overeat at...,15th Aug,High
3,test0004.jpg,No one:\nAbsolutely no one;\nRandom এনসিপি/বাগ...,Bangla,Mockery,Text,This meme satirizes pro-government student win...,বাগছাস / শিবির (BCL / Shibir),High
4,test0005.jpg,বন্ধু: বিকাশের নতুন আপডেট দেখসস? অনেক নতুন ফিচ...,Bangla,Ironic,Both,This meme humorously describes the feeling of ...,Low account balance,Low



Column Statistics:
   Image_name: 330/330 non-null
   extracted_text: 330/330 non-null
   Language: 330/330 non-null
   Humor: 330/330 non-null
   Metaphor: 330/330 non-null
   Meme_Explanation: 330/330 non-null
   Metaphor_Object: 330/330 non-null
   Political_Intensity: 330/330 non-null


### Verify Images
Check that all images referenced in the CSV exist in the image directory.

In [5]:
# Cell 5: Verify Images Exist
print("Verifying images...")

if os.path.exists(IMAGE_DIR):
    image_files = os.listdir(IMAGE_DIR)
    print(f"   Images in directory: {len(image_files)}")

    # Check matching
    csv_images = set(df['Image_name'].tolist())
    dir_images = set(image_files)

    matched = csv_images.intersection(dir_images)
    missing = csv_images - dir_images

    print(f"   Matched: {len(matched)}")
    print(f"   Missing: {len(missing)}")

    if missing:
        print(f"\nMissing images: {list(missing)[:10]}...")
else:
    print(f"Image directory not found: {IMAGE_DIR}")
    print("   Make sure the competition dataset is added to inputs.")

Verifying images...
   Images in directory: 330
   Matched: 330
   Missing: 0


### Chain-of-Thought Prompt
Define CoT prompt template using meme metadata (humor, metaphor, political intensity).

In [6]:
# Cell 6: Define Chain-of-Thought Prompt Template
# Uses underscore column names: Meme_Explanation, Metaphor_Object, Political_Intensity

def create_cot_prompt(row):
    """
    Create a Chain-of-Thought prompt using CSV data for the image.
    Column names with underscores as in predict.csv.
    """
    image_name = row.get('Image_name', 'Unknown')
    humor = row.get('Humor', 'Unknown')
    metaphor = row.get('Metaphor', 'Unknown')
    meme_explanation = row.get('Meme_Explanation', 'Not provided')
    metaphor_object = row.get('Metaphor_Object', 'Not provided')
    political_intensity = row.get('Political_Intensity', 'Unknown')
    
    prompt = f"""You are an expert meme analyst. Analyze this Bangla/Bengali meme and classify it as either "Political" or "NonPolitical".

## CONTEXT FROM ANALYSIS:
- **Image**: {image_name}
- **Humor Type**: {humor}
- **Metaphor Location**: {metaphor}
- **Meme Explanation**: {meme_explanation}
- **Key Metaphor Object**: {metaphor_object}
- **Political Intensity**: {political_intensity}

## CLASSIFICATION CRITERIA:

### Political Indicators (classify as "Political"):
- Political figures (Sheikh Hasina, Khaleda Zia, politicians)
- Political parties (BNP, Awami League, BCL, Shibir)
- Government, parliament, elections, voting
- Protests, hartals, political movements
- Corruption, political scandals
- Political Intensity is "High" strongly suggests Political

### Non-Political Indicators (classify as "NonPolitical"):
- Daily life humor (exams, jobs, relationships)
- Entertainment (cricket, movies, celebrities)
- Social situations without political context
- Political Intensity is "Low" strongly suggests NonPolitical

## DECISION LOGIC:
1. If Political_Intensity is "High" likely Political
2. If Political_Intensity is "Low" likely NonPolitical  
3. If Metaphor_Object contains political keywords Political
4. Otherwise analyze the meme context

## OUTPUT:
Based on the context and image, provide your classification:

FINAL_ANSWER: [Political or NonPolitical]"""
    
    return prompt

print("CoT prompt template defined!")
print("\nSample prompt preview:")
sample_prompt = create_cot_prompt(df.iloc[0])
print(sample_prompt[:800] + "...")

CoT prompt template defined!

Sample prompt preview:
You are an expert meme analyst. Analyze this Bangla/Bengali meme and classify it as either "Political" or "NonPolitical".

## CONTEXT FROM ANALYSIS:
- **Image**: test0001.jpg
- **Humor Type**: Ironic
- **Metaphor Location**: Both
- **Meme Explanation**: This meme satirizes Bangladeshi politician Nurul Haque Nur's iconic quote, 'I'm a bit disappointed,' which he famously said after the chaotic 2019 DUCSU (Dhaka University Central Students' Union) election. The humor comes from applying this massive understatement to the dramatic situation of losing a significant political race.
- **Key Metaphor Object**: Ducsu Election
- **Political Intensity**: High

## CLASSIFICATION CRITERIA:

### Political Indicators (classify as "Political"):
- Political figures (Sheikh Hasina, Khaleda Zia, politicians...


---
# STAGE 1: Initial Classification (Qwen2.5-VL)
### Load Qwen Model
Load Qwen2.5-VL-7B-Instruct with 4-bit NF4 quantization for T4 GPU (~8GB VRAM).

In [7]:
# Cell 7: Load Qwen2.5-VL Model (Optimized for Free Tier T4 GPU)
# USE GPU T4 x2 on Kaggle for best results!

def load_qwen_model():
    """
    Load Qwen2.5-VL-7B-Instruct with 4-bit quantization for Kaggle free tier.
    Uses ~8GB VRAM with 4-bit, works on T4 (16GB).
    """
    print("\n" + "="*60)
    print("LOADING QWEN2.5-VL-7B-INSTRUCT (4-bit Quantized)")
    print("="*60)
    
    # Use Qwen2.5-VL-7B model (publicly available)
    model_name = "Qwen/Qwen2.5-VL-7B-Instruct"
    
    print(f"\nLoading model: {model_name}")
    print("This may take 3-5 minutes with quantization...")
    
    # 4-bit quantization config for memory efficiency
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    
    # Load processor
    processor = AutoProcessor.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    print("Processor loaded!")
    
    # Load model with 4-bit quantization
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )
    print("Model loaded with 4-bit quantization!")
    
    # Print memory usage
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        print(f"GPU Memory Used: {allocated:.2f} GB")
    
    return model, processor


def unload_model(model, processor):
    """Unload model to free GPU memory."""
    print("\nUnloading model...")
    del model
    del processor
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Model unloaded and memory cleared!")

print("Model loading functions defined!")

Model loading functions defined!


### Prediction Function
Define single image prediction with OOM handling and label extraction.

In [8]:
# Cell 8: Define Prediction Function

def predict_single_image(model, processor, image_path, prompt, max_retries=2):
    """
    Predict classification for a single image with error handling.
    """
    for attempt in range(max_retries):
        try:
            # Load and resize image (reduce memory)
            image = Image.open(image_path).convert('RGB')
            # Resize if too large
            max_size = 512
            if max(image.size) > max_size:
                ratio = max_size / max(image.size)
                new_size = (int(image.size[0] * ratio), int(image.size[1] * ratio))
                image = image.resize(new_size, Image.LANCZOS)
            
            # Prepare messages in Qwen format
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image},
                        {"type": "text", "text": prompt}
                    ]
                }
            ]
            
            # Apply chat template
            text = processor.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
            )
            
            # Process inputs
            inputs = processor(
                text=[text],
                images=[image],
                padding=True,
                return_tensors="pt"
            ).to(model.device)
            
            # Generate response (short for efficiency)
            with torch.no_grad():
                generated_ids = model.generate(
                    **inputs,
                    max_new_tokens=256,
                    do_sample=False,
                    pad_token_id=processor.tokenizer.pad_token_id,
                )
            
            # Decode response
            generated_ids_trimmed = [
                out_ids[len(in_ids):] 
                for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            response = processor.batch_decode(
                generated_ids_trimmed, 
                skip_special_tokens=True
            )[0]
            
            # Extract final answer
            label = extract_label(response)
            
            # Clear intermediate tensors
            del inputs, generated_ids, generated_ids_trimmed
            
            return {
                'reasoning': response,
                'label': label,
                'status': 'success'
            }
            
        except torch.cuda.OutOfMemoryError:
            print(f"   OOM error, clearing cache (attempt {attempt+1})...")
            torch.cuda.empty_cache()
            gc.collect()
            if attempt == max_retries - 1:
                return {'reasoning': '', 'label': '', 'status': 'oom'}
        except Exception as e:
            print(f"   Error: {str(e)[:50]}")
            if attempt == max_retries - 1:
                return {'reasoning': '', 'label': '', 'status': 'failed'}
    
    return {'reasoning': '', 'label': '', 'status': 'failed'}

def extract_label(response):
    """
    Extract the final classification label from the model response.
    """
    response_upper = response.upper()
    
    # Look for explicit FINAL_ANSWER pattern
    if 'FINAL_ANSWER:' in response_upper:
        match = re.search(r'FINAL_ANSWER:\s*(POLITICAL|NONPOLITICAL|NON-POLITICAL)', response_upper)
        if match:
            label = match.group(1)
            return 'NonPolitical' if 'NON' in label else 'Political'
    
    # Fallback: Look for keywords in the last part of response
    last_part = response_upper[-300:]
    
    if 'NONPOLITICAL' in last_part or 'NON-POLITICAL' in last_part or 'NON POLITICAL' in last_part:
        return 'NonPolitical'
    elif 'POLITICAL' in last_part:
        return 'Political'
    
    # Last resort: empty (will be filled by fallback)
    return ''

print("Prediction functions defined!")

Prediction functions defined!


### Stage 1 Pipeline
Main prediction loop with checkpointing (saves every 10 images) and detailed reasoning output.

In [9]:
# Cell 9: Main Prediction Pipeline

def run_prediction_pipeline(df, image_dir, checkpoint_path, batch_save=10):
    """
    Run predictions on all images with checkpointing.
    Optimized for free-tier GPU with frequent saves.
    """
    print("\n" + "="*70)
    print("STARTING STAGE 1: INITIAL CLASSIFICATION")
    print("="*70)
    
    # Load model
    model, processor = load_qwen_model()
    
    # Prepare results DataFrame
    results_df = df.copy()
    if 'Predicted_Label' not in results_df.columns:
        results_df['Predicted_Label'] = ''
    if 'Reasoning' not in results_df.columns:
        results_df['Reasoning'] = ''
    
    # Check for existing checkpoint
    if os.path.exists(checkpoint_path):
        print(f"\nLoading checkpoint...")
        try:
            checkpoint_df = pd.read_csv(checkpoint_path)
            # Merge predictions
            for idx, row in checkpoint_df.iterrows():
                pred = row.get('Predicted_Label', '')
                if pd.notna(pred) and pred != '':
                    match_idx = results_df[results_df['Image_name'] == row['Image_name']].index
                    if len(match_idx) > 0:
                        results_df.loc[match_idx[0], 'Predicted_Label'] = pred
                        results_df.loc[match_idx[0], 'Reasoning'] = row.get('Reasoning', '')
            print(f"   Loaded checkpoint with existing predictions")
        except Exception as e:
            print(f"   Could not load checkpoint: {e}")
    
    # Find images to process
    to_process = results_df[
        (results_df['Predicted_Label'].isna()) | (results_df['Predicted_Label'] == '')
    ].index.tolist()
    total = len(to_process)
    
    print(f"\nStatus:")
    print(f"   Total images: {len(results_df)}")
    print(f"   Already done: {len(results_df) - total}")
    print(f"   To process: {total}")
    
    if total == 0:
        print("\nAll images already processed!")
        unload_model(model, processor)
        return results_df
    
    print("\n" + "-"*70)
    print("PROCESSING LOG")
    print("-"*70)
    
    success_count = 0
    failed_count = 0
    
    for i, idx in enumerate(to_process):
        row = results_df.loc[idx]
        image_name = row['Image_name']
        image_path = os.path.join(image_dir, image_name)
        
        # Progress
        print(f"\n[{i+1}/{total}] {image_name}")
        
        # Check image exists
        if not os.path.exists(image_path):
            print(f"   Image not found!")
            failed_count += 1
            continue
        
        # Create prompt
        prompt = create_cot_prompt(row)
        
        # Predict
        print(f"   Analyzing...")
        result = predict_single_image(model, processor, image_path, prompt)
        
        if result['status'] == 'success' and result['label']:
            results_df.loc[idx, 'Predicted_Label'] = result['label']
            results_df.loc[idx, 'Reasoning'] = str(result['reasoning'])[:500]
            print(f"   Predicted: {result['label']}")
            success_count += 1
        else:
            print(f"   Failed or empty prediction")
            failed_count += 1
        
        # Save checkpoint frequently
        if (i + 1) % batch_save == 0:
            print(f"\n   Saving checkpoint ({i+1}/{total} done)...")
            results_df.to_csv(checkpoint_path, index=False)
        
        # Clear cache every 20 images
        if (i + 1) % 20 == 0:
            torch.cuda.empty_cache()
            gc.collect()
    
    # Final save
    results_df.to_csv(checkpoint_path, index=False)
    
    print("\n" + "="*70)
    print("STAGE 1 COMPLETE!")
    print("="*70)
    print(f"   Successful: {success_count}")
    print(f"   Failed: {failed_count}")
    
    # Unload model
    unload_model(model, processor)
    
    return results_df

print("Prediction pipeline defined!")

Prediction pipeline defined!


### Run Stage 1
Execute Stage 1 classification on all images (~30-60 min on T4 GPU).

In [10]:
# Cell 10: Run Stage 1 Prediction Pipeline
# This cell processes all images. Takes ~30-60 minutes on T4 GPU.

print("Starting Stage 1: Initial Predictions")
print("This will take some time...")

results_df = run_prediction_pipeline(
    df=df,
    image_dir=IMAGE_DIR,
    checkpoint_path=CHECKPOINT_PATH,
    batch_save=10  # Save every 10 images for safety
)

Starting Stage 1: Initial Predictions
This will take some time...

STARTING STAGE 1: INITIAL CLASSIFICATION

LOADING QWEN2.5-VL-7B-INSTRUCT (4-bit Quantized)

Loading model: Qwen/Qwen2.5-VL-7B-Instruct
This may take 3-5 minutes with quantization...


preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

   Predicted: NonPolitical

[328/330] test0328.jpg
   Analyzing...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   Predicted: NonPolitical

[329/330] test0329.jpg
   Analyzing...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   Predicted: Political

[330/330] test0330.jpg
   Analyzing...
   Predicted: NonPolitical

   Saving checkpoint (330/330 done)...

STAGE 1 COMPLETE!
   Successful: 330
   Failed: 0

Unloading model...
Model unloaded and memory cleared!


### Stage 1 Status
Check prediction progress, label distribution, and save Stage 1 results.

In [11]:
# Cell 11: Check Stage 1 Prediction Status and Save

print("="*60)
print("STAGE 1 PREDICTION STATUS")
print("="*60)

total = len(results_df)
filled = results_df['Predicted_Label'].notna() & (results_df['Predicted_Label'] != '')
filled_count = filled.sum()
empty_count = total - filled_count

print(f"\nProgress:")
print(f"   Total: {total}")
print(f"   Predicted: {filled_count} ({filled_count/total*100:.1f}%)")
print(f"   Empty: {empty_count}")

if filled_count > 0:
    print(f"\nLabel Distribution:")
    label_counts = results_df[filled]['Predicted_Label'].value_counts()
    for label, count in label_counts.items():
        print(f"   {label}: {count} ({count/filled_count*100:.1f}%)")

# Show empty ones
if empty_count > 0:
    print(f"\nEmpty predictions ({empty_count}):")
    empty_rows = results_df[~filled]['Image_name'].tolist()
    for img in empty_rows[:10]:
        print(f"   {img}")
    if len(empty_rows) > 10:
        print(f"   ... and {len(empty_rows)-10} more")

# Save Stage 1 predictions
STAGE1_OUTPUT_PATH = f"{OUTPUT_DIR}/stage1_prediction.csv"
results_df.to_csv(STAGE1_OUTPUT_PATH, index=False)
print(f"\n✓ Stage 1 predictions saved to: {STAGE1_OUTPUT_PATH}")

STAGE 1 PREDICTION STATUS

Progress:
   Total: 330
   Predicted: 330 (100.0%)
   Empty: 0

Label Distribution:
   NonPolitical: 174 (52.7%)
   Political: 156 (47.3%)

✓ Stage 1 predictions saved to: /kaggle/working/stage1_prediction.csv


---
# STAGE 2: Keyword-Based Validation (Phi-3-Vision)
---

### Load Phi-3-Vision Model
Load Microsoft Phi-3-Vision-128k-Instruct for Stage 2 visual validation.

In [12]:
# Cell 12: Load Phi-3-Vision Model for Stage 2 Validation

def load_phi3_vision_model():
    """
    Load Microsoft Phi-3-Vision-128k-Instruct for Stage 2 keyword-based validation.
    """
    print("\n" + "="*60)
    print("LOADING PHI-3-VISION-128K-INSTRUCT FOR STAGE 2 VALIDATION")
    print("="*60)
    
    from transformers import AutoModelForCausalLM, AutoProcessor
    
    model_name = "microsoft/Phi-3-vision-128k-instruct"
    
    print(f"\nLoading model: {model_name}")
    print("This may take 3-5 minutes...")
    
    # Load processor
    processor = AutoProcessor.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    print("Processor loaded!")
    
    # Load model with float16 for T4 GPU
    # _attn_implementation='eager' is CRITICAL for T4 compatibility
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16,
        _attn_implementation="eager"
    )
    print("Model loaded!")
    
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        print(f"GPU Memory Used: {allocated:.2f} GB")
    
    return model, processor

print("Phi-3-Vision Stage 2 model loading function defined!")

Phi-3-Vision Stage 2 model loading function defined!


### Political Keywords and Validation Prompt
Define political/non-political keywords and Stage 2 validation prompt with knowledge base.

In [13]:
# Cell 13: Define Stage 2 Keywords and Simple Validation Prompt

# Keywords to improve Stage 1 predictions
POLITICAL_KEYWORDS = [
    # Specific terms
    'zohran', 'jahoran', 'জোহরান', 
    'bimpi', 'বিম্পি',
    'পদপ্রার্থী',  # Candidate/Nominee
    'aamjonota', 'Aamjonota', 'আমজনতা',  # Including Bangla
    'ncp', 'NCP', 'National Citizen Party',
    'জাতীয়তাবাদী ছাত্র',  # Nationalist student
    
    # Politicians
    'sheikh hasina', 'hasina', 'শেখ হাসিনা', 'হাসিনা',
    'khaleda zia', 'khaleda', 'খালেদা জিয়া', 'খালেদা',
    'tarek zia', 'tarique rahman', 'তারেক জিয়া', 'তারেক রহমান',
    'ershad', 'এরশাদ',
    'bangabandhu', 'বঙ্গবন্ধু', 'mujib', 'মুজিব',
    'zia', 'জিয়া', 'ziaur rahman', 'জিয়াউর রহমান',
    
    # Political parties & logos
    'awami league', 'আওয়ামী লীগ', 'awami', 'আওয়ামী',
    'bnp', 'বিএনপি', 'bangladesh nationalist party',
    'jatiya party', 'জাতীয় পার্টি',
    'jamaat', 'জামায়াত', 'shibir', 'শিবির',
    'bcl', 'ছাত্রলীগ', 'chhatra league',
    'chhatra dal', 'ছাত্রদল',
    
    # Political symbols/logos
    'boat logo', 'নৌকা', 'nouka',  # Awami League symbol
    'sheaf of paddy', 'ধানের শীষ',  # BNP symbol
    'plough', 'লাঙ্গল'  # Jatiya Party symbol
]

# Non-political keywords - if found, keep as NonPolitical
NON_POLITICAL_KEYWORDS = [
    'bkash', 'বিকাশ', 'bKash'
]

def check_keywords_in_text(text, keywords):
    """Check if any keyword is present in the text."""
    if pd.isna(text):
        return False, []
    
    text_lower = str(text).lower()
    found = [k for k in keywords if k.lower() in text_lower]
    return len(found) > 0, found

def check_non_political_keywords(text):
    """Check if non-political keywords are present."""
    if pd.isna(text):
        return False, []
    
    text_lower = str(text).lower()
    found = [k for k in NON_POLITICAL_KEYWORDS if k.lower() in text_lower]
    return len(found) > 0, found

def create_stage2_validation_prompt(row, previous_label):
    """Simple prompt using extracted_text and Language for validation."""
    
    # Get extracted text and language from the row
    extracted_text = row.get('extracted_text', '')
    if pd.isna(extracted_text):
        extracted_text = 'No text found'
    
    language = row.get('Language', '')
    if pd.isna(language):
        language = 'Unknown'
    
    prompt = f"""Look at this meme image carefully.

Current label: {previous_label}

EXTRACTED TEXT FROM MEME:
"{extracted_text}"

LANGUAGE: {language}

Check if you see ANY of these in the IMAGE or TEXT:

POLITICIANS:
- Sheikh Hasina, Khaleda Zia, Tarek Zia, Ershad, Bangabandhu/Mujib

POLITICAL PARTIES/LOGOS:
- Awami League (boat/নৌকা logo)
- BNP (sheaf of paddy/ধানের শীষ logo)
- Jamaat, Shibir, BCL, Chhatra Dal
- NCP, National Citizen Party, Aamjonota/আমজনতা

SPECIFIC KEYWORDS:
- zohran/jahoran (জোহরান), bimpi (বিম্পি)
- পদপ্রার্থী (candidate), জাতীয়তাবাদী ছাত্র (nationalist student)

NON-POLITICAL (keep as NonPolitical if found):
- bKash/বিকাশ (mobile banking)

If you find ANY politician, party logo, or political keyword in image OR extracted text → answer Political.
If you find bKash/বিকাশ → answer NonPolitical.
If you don't find any political content → keep current label.

FINAL_ANSWER: [Political or NonPolitical]"""
    
    return prompt

def predict_with_phi3(model, processor, image_path, prompt, max_retries=2):
    """
    Predict using Phi-3-Vision model.
    """
    for attempt in range(max_retries):
        try:
            # Load and resize image
            image = Image.open(image_path).convert('RGB')
            max_size = 512
            if max(image.size) > max_size:
                ratio = max_size / max(image.size)
                new_size = (int(image.size[0] * ratio), int(image.size[1] * ratio))
                image = image.resize(new_size, Image.LANCZOS)
            
            # Phi-3-Vision format with <|image_1|> placeholder
            user_prompt = f"<|image_1|>\n{prompt}"
            
            messages = [
                {"role": "user", "content": user_prompt}
            ]
            
            # Apply chat template
            text = processor.tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
            )
            
            # Process inputs
            inputs = processor(text, [image], return_tensors="pt").to(model.device)
            
            # Generate
            with torch.no_grad():
                generate_ids = model.generate(
                    **inputs,
                    max_new_tokens=200,
                    do_sample=False,
                    temperature=0.0
                )
            
            # Decode
            generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
            response = processor.batch_decode(
                generate_ids, 
                skip_special_tokens=True, 
                clean_up_tokenization_spaces=False
            )[0]
            
            # Extract label
            label = extract_label(response)
            
            # Clear tensors
            del inputs, generate_ids
            
            return {
                'reasoning': response,
                'label': label,
                'status': 'success'
            }
            
        except torch.cuda.OutOfMemoryError:
            print(f"   OOM error, clearing cache (attempt {attempt+1})...")
            torch.cuda.empty_cache()
            gc.collect()
            if attempt == max_retries - 1:
                return {'reasoning': '', 'label': '', 'status': 'oom'}
        except Exception as e:
            print(f"   Error: {str(e)[:50]}")
            if attempt == max_retries - 1:
                return {'reasoning': '', 'label': '', 'status': 'failed'}
    
    return {'reasoning': '', 'label': '', 'status': 'failed'}

print("Stage 2 keywords and prompt defined!")
print(f"Political keywords: {len(POLITICAL_KEYWORDS)}")
print(f"Non-political keywords: {len(NON_POLITICAL_KEYWORDS)}")

# Show sample of extracted_text and Language columns
print("\nSample extracted_text and Language from CSV:")
for i in range(min(3, len(df))):
    row = df.iloc[i]
    text = str(row.get('extracted_text', ''))[:80] if pd.notna(row.get('extracted_text')) else 'N/A'
    lang = row.get('Language', 'N/A')
    print(f"   {row['Image_name']}: Lang={lang}, Text={text}...")

Stage 2 keywords and prompt defined!
Political keywords: 60
Non-political keywords: 3

Sample extracted_text and Language from CSV:
   test0001.jpg: Lang=English, Text=*Looses Ducsu Election
- Any comment on the election?
- I'm a bid disappointed, ...
   test0002.jpg: Lang=Banglish, Text=kya matlab ke fue...
   test0003.jpg: Lang=Banglish, Text=SNACKS MUJIB DURING
15TH AUG SEHR!...


### Stage 2 Validation Pipeline
Two-phase validation: text-based keyword matching + visual validation with Phi-3-Vision.

In [14]:
# Cell 14: Run Stage 2 Keyword-Based Validation

def run_stage2_validation(results_df, image_dir, stage2_checkpoint_path):
    """
    Stage 2: Validate predictions using Phi-3-Vision with keyword focus.
    Only changes NonPolitical to Political if keywords/figures found.
    Also checks for non-political keywords to prevent false positives.
    """
    print("\n" + "="*70)
    print("STARTING STAGE 2: KEYWORD-BASED VALIDATION (PHI-3-VISION)")
    print("="*70)
    
    # First, do text-based keyword check (fast)
    print("\n[Phase 1] Text-based keyword detection...")
    keyword_matches = 0
    non_political_matches = 0
    
    for idx, row in results_df.iterrows():
        # Check all text columns
        all_text = ' '.join([
            str(row.get('extracted_text', '')),
            str(row.get('Language', '')),
            str(row.get('Meme_Explanation', '')),
            str(row.get('Metaphor_Object', '')),
            str(row.get('Humor', ''))
        ])
        
        # First check for non-political keywords (bkash, etc.)
        has_non_political, non_pol_found = check_non_political_keywords(all_text)
        
        if has_non_political and row['Predicted_Label'] == 'Political':
            # If bkash found and currently Political, change to NonPolitical
            results_df.loc[idx, 'Predicted_Label'] = 'NonPolitical'
            results_df.loc[idx, 'Reasoning'] = f'Stage2 non-political keyword: {non_pol_found[:3]}'
            non_political_matches += 1
            print(f"   {row['Image_name']}: Political -> NonPolitical (keywords: {non_pol_found[:3]})")
        elif row['Predicted_Label'] == 'NonPolitical' and not has_non_political:
            # Check for political keywords only if no non-political keywords found
            has_keywords, found = check_keywords_in_text(all_text, POLITICAL_KEYWORDS)
            
            if has_keywords:
                results_df.loc[idx, 'Predicted_Label'] = 'Political'
                results_df.loc[idx, 'Reasoning'] = f'Stage2 keyword match: {found[:3]}'
                keyword_matches += 1
                print(f"   {row['Image_name']}: NonPolitical -> Political (keywords: {found[:3]})")
    
    print(f"\nPhase 1 complete:")
    print(f"   NonPolitical -> Political: {keyword_matches}")
    print(f"   Political -> NonPolitical: {non_political_matches}")
    
    # Load Phi-3-Vision model for visual validation
    print("\n[Phase 2] Loading Phi-3-Vision for visual validation...")
    model, processor = load_phi3_vision_model()
    
    # Only validate NonPolitical predictions with model (to catch visual political content)
    to_validate = results_df[results_df['Predicted_Label'] == 'NonPolitical'].index.tolist()
    
    print(f"\nImages to validate with model: {len(to_validate)}")
    
    changes_count = 0
    
    for i, idx in enumerate(to_validate):
        row = results_df.loc[idx]
        image_name = row['Image_name']
        image_path = os.path.join(image_dir, image_name)
        previous_label = row['Predicted_Label']
        
        print(f"\n[{i+1}/{len(to_validate)}] {image_name}")
        print(f"   Current: {previous_label}")
        
        if not os.path.exists(image_path):
            print(f"   Image not found!")
            continue
        
        # Skip if non-political keyword found (already confirmed as non-political)
        all_text = ' '.join([
            str(row.get('extracted_text', '')),
            str(row.get('Meme_Explanation', '')),
            str(row.get('Metaphor_Object', ''))
        ])
        has_non_political, _ = check_non_political_keywords(all_text)
        if has_non_political:
            print(f"   Skipped: Contains non-political keyword (bkash/বিকাশ)")
            continue
        
        # Create validation prompt
        prompt = create_stage2_validation_prompt(row, previous_label)
        
        # Predict with Phi-3-Vision
        print(f"   Validating with Phi-3-Vision...")
        result = predict_with_phi3(model, processor, image_path, prompt)
        
        if result['status'] == 'success':
            new_label = result['label']
            
            # Check if political content was found
            found_political = 'FOUND_POLITICAL_CONTENT: YES' in result['reasoning'].upper() or \
                            ('YES' in result['reasoning'].upper().split('FOUND_POLITICAL_CONTENT:')[1].split('\n')[0] if 'FOUND_POLITICAL_CONTENT:' in result['reasoning'].upper() else False)
            
            if found_political or new_label == 'Political':
                results_df.loc[idx, 'Predicted_Label'] = 'Political'
                results_df.loc[idx, 'Reasoning'] = f'Stage2 visual validation: {result["reasoning"][:200]}'
                print(f"   Changed: NonPolitical -> Political (visual content detected)")
                changes_count += 1
            else:
                print(f"   Confirmed: NonPolitical (no political content found)")
        
        # Save checkpoint every 20 images
        if (i + 1) % 20 == 0:
            print(f"\n   Saving Stage 2 checkpoint...")
            results_df.to_csv(stage2_checkpoint_path, index=False)
            torch.cuda.empty_cache()
            gc.collect()
    
    # Final save
    results_df.to_csv(stage2_checkpoint_path, index=False)
    
    print("\n" + "="*70)
    print("STAGE 2 VALIDATION COMPLETE!")
    print("="*70)
    print(f"   Text-based Political changes: {keyword_matches}")
    print(f"   Text-based NonPolitical changes: {non_political_matches}")
    print(f"   Visual validation changes: {changes_count}")
    print(f"   Total changes: {keyword_matches + non_political_matches + changes_count}")
    
    # Unload model
    unload_model(model, processor)
    
    return results_df

print("Stage 2 validation pipeline defined!")

Stage 2 validation pipeline defined!


### Run Stage 2
Execute Stage 2 keyword-based validation on NonPolitical predictions.

In [15]:
# Cell 15: Execute Stage 2 Validation

print("Starting Stage 2: Keyword-Based Validation")
print("This will validate NonPolitical predictions for political content...\n")

results_df = run_stage2_validation(
    results_df=results_df,
    image_dir=IMAGE_DIR,
    stage2_checkpoint_path=STAGE2_CHECKPOINT_PATH
)

Starting Stage 2: Keyword-Based Validation
This will validate NonPolitical predictions for political content...


STARTING STAGE 2: KEYWORD-BASED VALIDATION (PHI-3-VISION)

[Phase 1] Text-based keyword detection...
   test0100.jpg: NonPolitical -> Political (keywords: ['পদপ্রার্থী'])
   test0104.jpg: NonPolitical -> Political (keywords: ['jahoran', 'জোহরান'])
   test0202.jpg: NonPolitical -> Political (keywords: ['sheikh hasina', 'hasina', 'শেখ হাসিনা'])
   test0236.jpg: NonPolitical -> Political (keywords: ['aamjonota', 'Aamjonota', 'আমজনতা'])
   test0270.jpg: NonPolitical -> Political (keywords: ['ncp', 'NCP', 'National Citizen Party'])
   test0302.jpg: NonPolitical -> Political (keywords: ['জামায়াত'])
   test0320.jpg: Political -> NonPolitical (keywords: ['বিকাশ'])

Phase 1 complete:
   NonPolitical -> Political: 6
   Political -> NonPolitical: 1

[Phase 2] Loading Phi-3-Vision for visual validation...

LOADING PHI-3-VISION-128K-INSTRUCT FOR STAGE 2 VALIDATION

Loading model: micro

preprocessor_config.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

processing_phi3_v.py: 0.00B [00:00, ?B/s]

image_processing_phi3_v.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- image_processing_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- processing_phi3_v.py
- image_processing_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:625: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


...

   Error: 'DynamicCache' object has no attribute 'get_max_le

[167/169] test0327.jpg
   Current: NonPolitical
   Validating with Phi-3-Vision...
   Error: 'DynamicCache' object has no attribute 'get_max_le
   Error: 'DynamicCache' object has no attribute 'get_max_le

[168/169] test0328.jpg
   Current: NonPolitical
   Validating with Phi-3-Vision...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   Error: 'DynamicCache' object has no attribute 'get_max_le
   Error: 'DynamicCache' object has no attribute 'get_max_le

[169/169] test0330.jpg
   Current: NonPolitical
   Validating with Phi-3-Vision...
   Error: 'DynamicCache' object has no attribute 'get_max_le


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   Error: 'DynamicCache' object has no attribute 'get_max_le

STAGE 2 VALIDATION COMPLETE!
   Text-based Political changes: 6
   Text-based NonPolitical changes: 1
   Visual validation changes: 0
   Total changes: 7

Unloading model...
Model unloaded and memory cleared!


---
# Final Submission
### Final Statistics
Display final label distribution and prediction summary after both stages.

In [16]:
# Cell 16: Final Statistics After Both Stages

print("="*60)
print("FINAL STATISTICS (AFTER STAGE 2)")
print("="*60)

total = len(results_df)
filled = (results_df['Predicted_Label'].notna()) & (results_df['Predicted_Label'] != '')
filled_count = filled.sum()

print(f"\nCompletion:")
print(f"   Total images: {total}")
print(f"   Predicted: {filled_count}")
print(f"   Completion rate: {filled_count/total*100:.1f}%")

print(f"\nFinal Label Distribution:")
label_counts = results_df['Predicted_Label'].value_counts()
for label, count in label_counts.items():
    if label and pd.notna(label):
        print(f"   {label}: {count} ({count/total*100:.1f}%)")

# Display sample results
print(f"\nSample Predictions:")
for idx, row in results_df.head(10).iterrows():
    print(f"   {row['Image_name']}: {row['Predicted_Label']}")

FINAL STATISTICS (AFTER STAGE 2)

Completion:
   Total images: 330
   Predicted: 330
   Completion rate: 100.0%

Final Label Distribution:
   NonPolitical: 169 (51.2%)
   Political: 161 (48.8%)

Sample Predictions:
   test0001.jpg: Political
   test0002.jpg: NonPolitical
   test0003.jpg: Political
   test0004.jpg: Political
   test0005.jpg: NonPolitical
   test0006.jpg: Political
   test0007.jpg: NonPolitical
   test0008.jpg: Political
   test0009.jpg: NonPolitical
   test0010.jpg: NonPolitical


### Handle Empty Predictions
Apply fallback logic for any remaining empty predictions using political intensity.

In [17]:
# Cell 17: Handle Any Remaining Empty Predictions

empty_mask = (results_df['Predicted_Label'].isna()) | (results_df['Predicted_Label'] == '')
empty_count = empty_mask.sum()

if empty_count > 0:
    print(f"Warning: {empty_count} predictions still empty. Applying fallback logic...")
    
    for idx in results_df[empty_mask].index:
        row = results_df.loc[idx]
        
        # Get context
        intensity = str(row.get('Political_Intensity', '')).lower().strip()
        metaphor_obj = str(row.get('Metaphor_Object', '')).lower()
        explanation = str(row.get('Meme_Explanation', '')).lower()
        
        # Check for keywords in all text
        all_text = f"{metaphor_obj} {explanation}"
        has_keywords, _ = check_keywords_in_text(all_text, POLITICAL_KEYWORDS)
        
        # Decision logic
        if has_keywords or intensity == 'high':
            label = 'Political'
        elif intensity == 'low':
            label = 'NonPolitical'
        else:
            label = 'NonPolitical'
        
        results_df.loc[idx, 'Predicted_Label'] = label
        results_df.loc[idx, 'Reasoning'] = f'Fallback: intensity={intensity}'
        print(f"   {row['Image_name']}: {label} (fallback)")
    
    print(f"\nAll empty predictions filled!")
else:
    print("No empty predictions!")

No empty predictions!


### Create Submission CSV
Generate final submission file with Image_name and Label columns.

In [18]:
# Cell 18: Create Final Submission CSV

print("="*60)
print("CREATING FINAL SUBMISSION FILE")
print("="*60)

# Create submission DataFrame
submission_df = results_df[['Image_name', 'Predicted_Label']].copy()
submission_df.columns = ['Image_name', 'Label']

# Ensure no empty labels
empty_labels = submission_df['Label'].isna() | (submission_df['Label'] == '')
if empty_labels.sum() > 0:
    print(f"Warning: Found {empty_labels.sum()} empty labels, filling with NonPolitical...")
    submission_df.loc[empty_labels, 'Label'] = 'NonPolitical'

# Save submission
submission_df.to_csv(SUBMISSION_PATH, index=False)

print(f"\nSubmission saved to: {SUBMISSION_PATH}")
print(f"Total records: {len(submission_df)}")
print(f"Columns: {submission_df.columns.tolist()}")

print(f"\nFinal Label Distribution:")
print(submission_df['Label'].value_counts())

print(f"\nFirst 10 rows:")
display(submission_df.head(10))

print(f"\nLast 10 rows:")
display(submission_df.tail(10))

CREATING FINAL SUBMISSION FILE

Submission saved to: /kaggle/working/submission.csv
Total records: 330
Columns: ['Image_name', 'Label']

Final Label Distribution:
Label
NonPolitical    169
Political       161
Name: count, dtype: int64

First 10 rows:


,Image_name,Label
0,test0001.jpg,Political
1,test0002.jpg,NonPolitical
2,test0003.jpg,Political
3,test0004.jpg,Political
4,test0005.jpg,NonPolitical
5,test0006.jpg,Political
6,test0007.jpg,NonPolitical
7,test0008.jpg,Political
8,test0009.jpg,NonPolitical
9,test0010.jpg,NonPolitical



Last 10 rows:


,Image_name,Label
320,test0321.jpg,NonPolitical
321,test0322.jpg,Political
322,test0323.jpg,Political
323,test0324.jpg,Political
324,test0325.jpg,NonPolitical
325,test0326.jpg,NonPolitical
326,test0327.jpg,NonPolitical
327,test0328.jpg,NonPolitical
328,test0329.jpg,Political
329,test0330.jpg,NonPolitical


### Verify Submission
Validate submission format, check for missing/invalid labels.

In [19]:
# Cell 19: Verify Submission Format

print("="*60)
print("SUBMISSION VERIFICATION")
print("="*60)

# Reload and verify
verify_df = pd.read_csv(SUBMISSION_PATH)

print(f"\nFile: {SUBMISSION_PATH}")
print(f"Shape: {verify_df.shape}")
print(f"Columns: {verify_df.columns.tolist()}")
print(f"\nValue counts:")
print(verify_df['Label'].value_counts())

# Check for issues
issues = []

if len(verify_df.columns) != 2:
    issues.append(f"Expected 2 columns, got {len(verify_df.columns)}")

if 'Image_name' not in verify_df.columns:
    issues.append("Missing 'Image_name' column")

if 'Label' not in verify_df.columns:
    issues.append("Missing 'Label' column")

empty_labels = verify_df['Label'].isna().sum() + (verify_df['Label'] == '').sum()
if empty_labels > 0:
    issues.append(f"{empty_labels} empty labels")

invalid_labels = ~verify_df['Label'].isin(['Political', 'NonPolitical'])
if invalid_labels.sum() > 0:
    issues.append(f"{invalid_labels.sum()} invalid labels")
    print(f"\nInvalid labels found:")
    print(verify_df[invalid_labels]['Label'].unique())

if issues:
    print(f"\nIssues found:")
    for issue in issues:
        print(f"   {issue}")
else:
    print(f"\nSubmission file is valid!")
    print("Ready for submission!")

SUBMISSION VERIFICATION

File: /kaggle/working/submission.csv
Shape: (330, 2)
Columns: ['Image_name', 'Label']

Value counts:
Label
NonPolitical    169
Political       161
Name: count, dtype: int64

Submission file is valid!
Ready for submission!


### Save Full Results
Save complete results with reasoning from both stages for analysis.

In [20]:
# Cell 20: Save Full Results with Reasoning

full_results_path = f"{OUTPUT_DIR}/full_results_2stage.csv"
results_df.to_csv(full_results_path, index=False)

print(f"Full results saved to: {full_results_path}")
print(f"Contains: {len(results_df)} records with reasoning from both stages")

Full results saved to: /kaggle/working/full_results_2stage.csv
Contains: 330 records with reasoning from both stages


### Cleanup (Optional)
Uncomment to clear /kaggle/working directory if needed.

In [21]:
# import shutil
# import os

# work_dir = "/kaggle/working"

# # Remove everything inside /kaggle/working
# for item in os.listdir(work_dir):
#     item_path = os.path.join(work_dir, item)
#     try:
#         if os.path.isfile(item_path) or os.path.islink(item_path):
#             os.unlink(item_path)   # delete file or symlink
#         elif os.path.isdir(item_path):
#             shutil.rmtree(item_path)   # delete folder
#     except Exception as e:
#         print(f"Could not delete {item_path}: {e}")

# print("✔ /kaggle/working directory cleaned successfully.")

---

## Done!

### Two-Stage Classification Complete:

**Stage 1**: Qwen2.5-VL-7B initial classification
- General political/non-political classification
- Uses meme context and metadata

**Stage 2**: Phi-3-Vision-128k-Instruct keyword validation
- **Knowledge base injected** for political content detection
- Text-based keyword detection in all columns
- Visual validation for political figures and logos
- Only changes NonPolitical → Political when political content found
- Changes Political → NonPolitical when non-political keywords found (e.g., bKash)

### Files Generated:
1. **`/kaggle/working/submission.csv`** - Final submission (Image_name, Label)
2. **`/kaggle/working/prediction_checkpoint.csv`** - Stage 1 results
3. **`/kaggle/working/stage2_checkpoint.csv`** - Stage 2 validation results
4. **`/kaggle/working/full_results_2stage.csv`** - Complete results with reasoning

### Political Keywords Used:

**Specific Terms:**
- zohran, jahoran (জোহরান)
- bimpi (বিম্পি)
- পদপ্রার্থী (Candidate/Nominee)
- Aamjonota (আমজনতা)
- NCP, National Citizen Party
- জাতীয়তাবাদী ছাত্র (Nationalist student)

**Politicians:**
- Sheikh Hasina (শেখ হাসিনা), Hasina (হাসিনা)
- Khaleda Zia (খালেদা জিয়া), Khaleda (খালেদা)
- Tarek Zia, Tarique Rahman (তারেক জিয়া, তারেক রহমান)
- Ershad (এরশাদ)
- Bangabandhu (বঙ্গবন্ধু), Mujib (মুজিব)
- Zia (জিয়া), Ziaur Rahman (জিয়াউর রহমান)

**Political Parties:**
- Awami League (আওয়ামী লীগ)
- BNP (বিএনপি), Bangladesh Nationalist Party
- Jatiya Party (জাতীয় পার্টি)
- Jamaat (জামায়াত), Shibir (শিবির)
- BCL (ছাত্রলীগ), Chhatra League
- Chhatra Dal (ছাত্রদল)

**Political Symbols:**
- নৌকা / nouka / boat logo (Awami League)
- ধানের শীষ / sheaf of paddy (BNP)
- লাঙ্গল / plough (Jatiya Party)

---